In [19]:
import pandas as pd
import numpy as np
import datetime as dt

In [15]:
df=pd.read_csv('Weather_BTS.csv',index_col=0)
df=df[df['DEP_DELAY'].notna()]
df=df[df['DEP_DELAY']>=-20]
df=df[df["DEP_DELAY"]<=200]
delay=[]
for i,j in df['DEP_DELAY'].iteritems():
    if(j>15):
        delay.append(1)
    else:
        delay.append(0)
df['DELAY']=delay
df=df.drop(['PGTM','WDF2','WDF5',"WSF5",'TMAX', 'TMIN','DEP_DELAY',],axis=1)
df['DEP_TIME']=df['DEP_TIME']//100
df.columns

Index(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'OP_UNIQUE_CARRIER', 'DEP_TIME',
       'DATE', 'STATION', 'NAME', 'AWND', 'PRCP', 'SNOW', 'SNWD', 'TAVG',
       'WSF2', 'WT01', 'WT02', 'WT03', 'WT04', 'WT06', 'WT08', 'WT09',
       'DELAY'],
      dtype='object')

In [16]:
a=[1,2,3,4,6,8,9]
for i in a:
    s='WT'+str(0)+str(i)
    a=np.where(df[s].isna(),0,1)
    df[s]=a
df.columns

Index(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'OP_UNIQUE_CARRIER', 'DEP_TIME',
       'DATE', 'STATION', 'NAME', 'AWND', 'PRCP', 'SNOW', 'SNWD', 'TAVG',
       'WSF2', 'WT01', 'WT02', 'WT03', 'WT04', 'WT06', 'WT08', 'WT09',
       'DELAY'],
      dtype='object')

In [22]:
holidays = pd.read_csv("US Bank holidays.csv",names=['Date','Holiday'])
holidays=holidays[(holidays['Date']>='2013-08') & (holidays['Date']<='2018-08')]
holidays['label']=1
holidays=holidays.reset_index(drop=True)
holidays['Date']=pd.to_datetime(holidays['Date'], format = '%Y-%m-%d')
holidays=holidays.drop('Holiday',axis=1)
for i,row in holidays.iterrows():
    holidays=holidays.append({'Date':row['Date']+dt.timedelta(days=1),'label':0.67},ignore_index=True)
    holidays=holidays.append({'Date':row['Date']+dt.timedelta(days=2),'label':0.33},ignore_index=True)
    holidays=holidays.append({'Date':row['Date']-dt.timedelta(days=1),'label':0.67},ignore_index=True)
    holidays=holidays.append({'Date':row['Date']-dt.timedelta(days=2),'label':0.33},ignore_index=True)
holidays['DATE']=holidays["Date"]
holidays=holidays.drop('Date',axis=1)
df['DATE']=pd.to_datetime(df['DATE'], format = '%Y-%m-%d')

In [29]:
merged_df=df.merge(holidays,on='DATE',how='outer')
merged_df['HOLIDAY']=merged_df['label'].fillna(0)
merged_df=merged_df.drop(['label','DATE','STATION'],axis=1)
merged_df.columns

Index(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'OP_UNIQUE_CARRIER', 'DEP_TIME',
       'NAME', 'AWND', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'WSF2', 'WT01', 'WT02',
       'WT03', 'WT04', 'WT06', 'WT08', 'WT09', 'DELAY', 'HOLIDAY'],
      dtype='object')

In [75]:
carrier=pd.get_dummies(merged_df['OP_UNIQUE_CARRIER'])
month=pd.get_dummies(merged_df['MONTH'])
month=month.rename({1:'JAN',2:'FEB',3:'MAR',4:'APR',5:'MAY',6:'JUN',7:'JUL',8:'AUG',9:'SEP',10:'OCT',11:'NOV',12:'DEC'},axis=1)
time=pd.get_dummies(merged_df['DEP_TIME'])
time=time.rename(dict((i,'h'+str(int(i))) for i in time.columns.values),axis=1)
time['h0']=time['h0']+time['h24']
time=time.drop('h24',axis=1)
merged=merged_df.merge(carrier,on=merged_df.index)
merged=merged.drop('key_0',axis=1)
merged=merged.merge(time,on=merged.index)
merged=merged.drop('key_0',axis=1)
merged=merged.merge(month,on=merged.index)
merged=merged.drop(['OP_UNIQUE_CARRIER','DEP_TIME','MONTH','WSF2','key_0','NAME','YEAR'],axis=1)
merged=merged.rename(columns={'DAY_OF_MONTH':'DAY','DELAY':'Y','AWND':'WIND_SPEED','PRCP':'RAIN','SNWD':'SNOW_DEPTH','WT01':'FOG','WT02':'HEAVY_FOG','WT03':'THUNDER','WT04':'HAIL','WT06':'GLAZE','WT08':'HAZE','WT09':'BLOWING_SNOW','holiday':'HOLIDAY'})
merged=merged.drop('DAY',axis=1)

In [76]:
merged.columns

Index(['WIND_SPEED', 'RAIN', 'SNOW', 'SNOW_DEPTH', 'TAVG', 'FOG', 'HEAVY_FOG',
       'THUNDER', 'HAIL', 'GLAZE', 'HAZE', 'BLOWING_SNOW', 'Y', 'HOLIDAY',
       '9E', 'AA', 'AS', 'B6', 'DL', 'EV', 'HA', 'MQ', 'OH', 'OO', 'UA', 'US',
       'VX', 'YX', 'h0', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8', 'h9',
       'h10', 'h11', 'h12', 'h13', 'h14', 'h15', 'h16', 'h17', 'h18', 'h19',
       'h20', 'h21', 'h22', 'h23', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN',
       'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'],
      dtype='object')

In [77]:
merged.to_csv('Final.csv')